### Method which combined global/local percentil score 

In [102]:
import pandas as pd
import numpy as np

In [104]:
dataset1=pd.read_csv("../dataset_unified.csv", sep=";")
dataset1.head()

,company,date,quarter,country,ROA,ROE,debt_to_equity,current_ratio,net_margin,revenue_growth,cash_ratio,inflation_YoY,gdp_growth_rate,interest_rate
0,Banco Santander,2024-09-30,2024-Q3,Spain,NaN,NaN,"16,15","0,33",NaN,"6,2414",NaN,"0,0217","0,0080","0,0365"
1,Banco Santander,2024-06-30,2024-Q2,Spain,"0,63","0,109","16,23","0,23","0,0906","-0,8712","0,23","0,0346","0,0080","0,0425"
2,Banco Santander,2024-03-31,2024-Q1,Spain,"0,6","0,1043","16,14","0,24","0,087","-0,6185","0,24","0,0314","0,0100","0,0450"
3,Banco Santander,2023-12-31,2023-Q4,Spain,"0,59","0,1035","16,24","0,35","0,0886","22,1927","0,34","0,0327","0,0070","0,0450"
4,Banco Santander,2023-09-30,2023-Q3,Spain,"0,55","0,0968","16,66","0,31","0,1044","0,0201","0,31","0,0282","0,0070","0,0450"


In [105]:
indicators_to_clean = [
    'ROA', 'ROE', 'net_margin', 'current_ratio',
    'cash_ratio', 'debt_to_equity', 'revenue_growth',
    'inflation_YoY', 'gdp_growth_rate', 'interest_rate'
]

# STEP 1: Clean all comma-based numbers → dots, then convert to numeric
for col in indicators_to_clean:
    dataset1[col] = (
        dataset1[col]
        .astype(str)
        .str.replace(',', '.', regex=False)     # Convert commas to dots
        .replace('nan', np.nan)                 # Replace string 'nan' with real NaN
    )
    dataset1[col] = pd.to_numeric(dataset1[col], errors='coerce')  # Convert to float

print(dataset1[indicators_to_clean].dtypes)

ROA                float64
ROE                float64
net_margin         float64
current_ratio      float64
cash_ratio         float64
debt_to_equity     float64
revenue_growth     float64
inflation_YoY      float64
gdp_growth_rate    float64
interest_rate      float64
dtype: object


In [106]:
# Fonction pour appliquer le percentile local (rank normalisé)
def compute_local_percentile(df, column):
    return df.groupby('company')[column].rank(pct=True)

# Appliquer aux colonnes clés
dataset1['ROA_pct'] = compute_local_percentile(dataset1, 'ROA')
dataset1['ROE_pct'] = compute_local_percentile(dataset1, 'ROE')
dataset1['net_margin_pct'] = compute_local_percentile(dataset1, 'net_margin')
dataset1['current_ratio_pct'] = compute_local_percentile(dataset1, 'current_ratio')
dataset1['cash_ratio_pct'] = compute_local_percentile(dataset1, 'cash_ratio')
dataset1['debt_to_equity_pct'] = compute_local_percentile(dataset1, 'debt_to_equity')


In [107]:
# Score 1: Profitabilité
dataset1['score_profitability_local'] = dataset1[['ROA_pct', 'ROE_pct', 'net_margin_pct']].mean(axis=1)

# Score 2: Liquidité
dataset1['score_liquidity_local'] = dataset1[['current_ratio_pct', 'cash_ratio_pct']].mean(axis=1)

# Score 3: Solvabilité — attention, plus debt_to_equity est bas, mieux c’est
dataset1['score_solvency_local'] = 1 - dataset1['debt_to_equity_pct']

# Score 4: Profitabilité ajustée à l'endettement
dataset1['inv_debt_pct'] = 1 - dataset1['debt_to_equity_pct']
dataset1['score_leverage_adjusted_local'] = dataset1[['ROE_pct', 'inv_debt_pct']].mean(axis=1)


In [ ]:
cols = ['company', 'quarter', 'score_profitability_local', 'score_liquidity_local', 'score_solvency_local', 'score_leverage_adjusted_local']
print(dataset1[cols].head(10))


           company  quarter  score_profitability_local  score_liquidity_local  score_solvency_local  score_leverage_adjusted_local
0  Banco Santander  2024-Q3                        NaN               0.255319              0.191489                       0.021277
1  Banco Santander  2024-Q2                   0.913043               0.063830              0.170213                       0.659574
2  Banco Santander  2024-Q1                   0.739130               0.127660              0.212766                       0.702128
3  Banco Santander  2023-Q4                   0.739130               0.382979              0.148936                       0.574468
4  Banco Santander  2023-Q3                   0.891304               0.223404              0.085106                       0.446809
5  Banco Santander  2023-Q2                   0.695652               0.127660              0.127660                       0.425532
6  Banco Santander  2023-Q1                   0.630435               0.127660      

In [108]:
def compute_global_percentile(df, column):
    return df[column].rank(pct=True)

# Appliquer le percentile global
dataset1['ROA_pct_global'] = compute_global_percentile(dataset1, 'ROA')
dataset1['ROE_pct_global'] = compute_global_percentile(dataset1, 'ROE')
dataset1['net_margin_pct_global'] = compute_global_percentile(dataset1, 'net_margin')
dataset1['current_ratio_pct_global'] = compute_global_percentile(dataset1, 'current_ratio')
dataset1['cash_ratio_pct_global'] = compute_global_percentile(dataset1, 'cash_ratio')
dataset1['debt_to_equity_pct_global'] = compute_global_percentile(dataset1, 'debt_to_equity')

dataset1['debt_to_equity_pct_global'] = 1- dataset1['debt_to_equity_pct_global']  # Inverser pour la solvabilité

In [109]:
# Score global
dataset1['score_profitabilty_global']= dataset1[['ROA_pct_global','ROE_pct_global','net_margin_pct_global']].mean(axis=1)
dataset1['score_liquidity_global'] = dataset1[['current_ratio_pct_global','cash_ratio_pct_global']].mean(axis=1)
dataset1['score_solvency_global'] = dataset1['debt_to_equity_pct_global']
dataset1['score_leverage_adjusted_global'] = dataset1[['ROE_pct_global', 'debt_to_equity_pct_global']].mean(axis=1)



In [ ]:
cols=['company', 'quarter', 'score_profitabilty_global', 'score_liquidity_global', 'score_solvency_global', 'score_leverage_adjusted_global']
print(dataset1[cols].head(10))

           company  quarter  score_profitabilty_global  score_liquidity_global  score_solvency_global  score_leverage_adjusted_global
0  Banco Santander  2024-Q3                        NaN                0.061538               0.343590                        0.343590
1  Banco Santander  2024-Q2                   0.550775                0.041546               0.338462                        0.607619
2  Banco Santander  2024-Q1                   0.523870                0.057051               0.348718                        0.600899
3  Banco Santander  2023-Q4                   0.519894                0.220793               0.333333                        0.590837
4  Banco Santander  2023-Q3                   0.510348                0.142808               0.317949                        0.559448
5  Banco Santander  2023-Q2                   0.480298                0.057051               0.328205                        0.534956
6  Banco Santander  2023-Q1                   0.459738        

In [110]:
# Remplace 'dataset1' par le nom de ta DataFrame si différent
dataset1.to_csv("dataset1_complet.csv", index=False)


In [ ]:
dataset1.loc[dataset1['score_solvency_global'].idxmax()]


company                           JP Morgan Chase
date                                   2015-12-31
quarter                                   2015-Q4
country                                       USA
ROA                                          0.92
                                       ...       
trend_liquidity_global                  -0.119898
trend_solvency_local                     0.076923
trend_solvency_global                    0.082051
trend_leverage_adjusted_local            0.163462
trend_leverage_adjusted_global           0.109813
Name: 198, Length: 84, dtype: object

In [ ]:
print(dataset1.columns.tolist())


['company', 'date', 'quarter', 'country', 'ROA', 'ROE', 'debt_to_equity', 'current_ratio', 'net_margin', 'revenue_growth', 'cash_ratio', 'inflation_YoY', 'gdp_growth_rate', 'interest_rate', 'ROA_pct', 'ROE_pct', 'net_margin_pct', 'current_ratio_pct', 'cash_ratio_pct', 'debt_to_equity_pct', 'score_profitability', 'score_liquidity', 'score_solvency', 'inv_debt_pct', 'score_leverage_adjusted', 'ROA_pct_global', 'ROE_pct_global', 'net_margin_pct_global', 'current_ratio_pct_global', 'cash_ratio_pct_global', 'debt_to_equity_pct_global', 'score_profitabilty_global', 'score_liquidity_global', 'score_solvency_global', 'score_leverage_adjusted_global', 'score_profitability_global', 'score_profitability_local', 'score_liquidity_local', 'score_solvency_local', 'score_leverage_adjusted_local', 'score_profitability_local_flag', 'score_liquidity_local_flag', 'score_solvency_local_flag', 'score_leverage_adjusted_local_flag', 'score_profitability_global_global_flag', 'score_liquidity_global_global_flag

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Seuils top/bas
BOTTOM = 0.10
TOP = 0.90

# Fonction de classification simple
def classify_percentile(p):
    if pd.isna(p):
        return 'Missing'
    elif p <= BOTTOM:
        return 'Bottom 10%'
    elif p >= TOP:
        return 'Top 10%'
    else:
        return 'Middle'

# Appliquer les flags de percentiles à chaque score
for col in ['score_profitability', 'score_liquidity', 'score_solvency', 'score_leverage_adjusted']:
    dataset1[f'{col}_global_flag'] = dataset1[f'{col}_global'].apply(classify_percentile)
    dataset1[f'{col}_local_flag'] = dataset1[f'{col}_local'].apply(classify_percentile)

# Fonction pour interpréter les anomalies
def interpret_combined_flags(row):
    alerts = []
    metrics = ['score_profitability', 'score_liquidity', 'score_solvency', 'score_leverage_adjusted']
    for metric in metrics:
        g_flag = row.get(f'{metric}_global_flag')
        l_flag = row.get(f'{metric}_local_flag')

        if g_flag == 'Bottom 10%' and l_flag == 'Bottom 10%':
            alerts.append(f"{metric.replace('score_', '').capitalize()}: Global & Local Bottom → ⚠️")
        elif g_flag == 'Top 10%' and l_flag == 'Top 10%':
            alerts.append(f"{metric.replace('score_', '').capitalize()}: Strong signal ✅")
        elif g_flag == 'Bottom 10%' and l_flag == 'Top 10%':
            alerts.append(f"{metric.replace('score_', '').capitalize()}: Outperforms peers ↗️")
        elif g_flag == 'Top 10%' and l_flag == 'Bottom 10%':
            alerts.append(f"{metric.replace('score_', '').capitalize()}: Underperforms sector ↘️")
    return "; ".join(alerts) if alerts else "No major divergence"

dataset1['Trend / Risk Summary'] = dataset1.apply(interpret_combined_flags, axis=1)

# Widget de sélection de société
company_dropdown = widgets.Dropdown(
    options=sorted(dataset1['company'].unique()),
    description='Company:',
    value='Banco Santander'
)

output = widgets.Output()

def update_table(change):
    with output:
        clear_output()
        company = change['new']
        df = dataset1[dataset1['company'] == company].copy()

        # Colonnes à afficher
        display(df[[
            'quarter',
            'score_profitability', 'score_profitability_local_flag', 'score_profitability_global_flag',
            'score_liquidity', 'score_liquidity_local_flag', 'score_liquidity_global_flag',
            'score_solvency', 'score_solvency_local_flag', 'score_solvency_global_flag',
            'score_leverage_adjusted', 'score_leverage_adjusted_local_flag', 'score_leverage_adjusted_global_flag',
            'Trend / Risk Summary'
        ]].reset_index(drop=True))

# Affichage
company_dropdown.observe(update_table, names='value')
display(company_dropdown, output)
update_table({'new': company_dropdown.value})


Dropdown(description='Company:', index=1, options=('BNP Paribas', 'Banco Santander', 'Crédit Agricole', 'HSBC'…

Output()

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Définir les seuils
thresholds = {
    'low': 0.1,
    'high': 0.9
}

# Classer le percentile
def flag_percentile(p):
    if pd.isna(p): return 'Missing'
    elif p <= thresholds['low']: return 'Bottom 10%'
    elif p >= thresholds['high']: return 'Top 10%'
    else: return 'Middle'

# Appliquer flags
for col in ['score_profitability', 'score_liquidity', 'score_solvency', 'score_leverage_adjusted']:
    dataset1[f'{col}_global_flag'] = dataset1[f'{col}_global'].apply(flag_percentile)
    dataset1[f'{col}_local_flag'] = dataset1[f'{col}_local'].apply(flag_percentile)

# Revenue growth : alerte simple
def revenue_alert(x):
    if pd.isna(x): return "Unknown"
    elif x < -0.1: return "Rev ↓"
    elif x > 0.1: return "Rev ↑"
    else: return "Stable"

dataset1['revenue_alert'] = dataset1['revenue_growth'].apply(revenue_alert)

# Résumé alerte
def alert_summary(row):
    reds, greens, stables = [], [], []
    for col, name in [
        ('score_profitability', 'Profitability'),
        ('score_liquidity', 'Liquidity'),
        ('score_solvency', 'Solvency'),
        ('score_leverage_adjusted', 'Adj. Leverage')
    ]:
        val = row.get(col)
        if pd.isna(val): continue
        if val < thresholds['low']:
            reds.append(name)
        elif val > thresholds['high']:
            greens.append(name)
        else:
            stables.append(name)
    parts = []
    if reds:
        parts.append("Red (" + ", ".join(reds) + ")")
    if greens:
        parts.append("Green (" + ", ".join(greens) + ")")
    if not reds and not greens and stables:
        parts.append("Stable (" + ", ".join(stables) + ")")
    if not parts:
        parts.append("Missing")
    if row['revenue_alert'] in ['Rev ↓', 'Rev ↑']:
        parts.append(row['revenue_alert'])
    return ", ".join(parts)

dataset1['Alert Summary'] = dataset1.apply(alert_summary, axis=1)

# Statut global
def global_status(row):
    indicators = ['score_profitability', 'score_liquidity', 'score_solvency', 'score_leverage_adjusted']
    red = sum(row[col] < thresholds['low'] for col in indicators if pd.notna(row[col]))
    green = sum(row[col] > thresholds['high'] for col in indicators if pd.notna(row[col]))
    leverage = row['score_leverage_adjusted']
    rev = row['revenue_alert']

    if leverage is not None and leverage < thresholds['low']:
        return "Leveraged Risk"
    if leverage is not None and leverage > thresholds['high'] and red == 0 and rev == "Rev ↑":
        return "Excellent Health"
    if leverage is not None and leverage > thresholds['high'] and red == 0:
        return "Strong Capital Efficiency"
    if red >= 3:
        return "Critical Risk"
    if red == 2:
        return "Danger"
    if green >= 2 and red == 0 and leverage <= thresholds['high']:
        return "Strong"
    if green >= 2 and red <= 1 or (green >= 1 and red == 0):
        return "Good signal"
    if red == green and red > 0:
        return "Mixed Risk"
    if red == 1 and green == 0:
        return "Caution"
    if all(0.2 <= row[col] <= 0.8 for col in indicators if pd.notna(row[col])):
        return "Stable"
    return "Watch"

dataset1["Overall Status"] = dataset1.apply(global_status, axis=1)

# Widget pour sélection de société
dropdown = widgets.Dropdown(
    options=sorted(dataset1['company'].dropna().unique()),
    description='Company:',
    value=dataset1['company'].dropna().unique()[0]
)

output = widgets.Output()

def display_summary(change):
    with output:
        clear_output()
        df = dataset1[dataset1['company'] == change['new']].copy()
        df = df[[
            'quarter',
            'score_profitability', 'score_profitability_local_flag', 'score_profitability_global_flag',
            'score_liquidity', 'score_liquidity_local_flag', 'score_liquidity_global_flag',
            'score_solvency', 'score_solvency_local_flag', 'score_solvency_global_flag',
            'score_leverage_adjusted', 'score_leverage_adjusted_local_flag', 'score_leverage_adjusted_global_flag',
            'revenue_alert', 'Alert Summary', 'Overall Status'
        ]].reset_index(drop=True)
        display(df)

# Lancer
dropdown.observe(display_summary, names='value')
display(dropdown, output)
display_summary({'new': dropdown.value})


Dropdown(description='Company:', options=('BNP Paribas', 'Banco Santander', 'Crédit Agricole', 'HSBC', 'JP Mor…

Output()

In [ ]:
# Thresholds
low_threshold = 0.1
high_threshold = 0.9

# Helper: flag anomalies
def flag_anomalies(df, score_col, suffix):
    df[f'is_low_{suffix}'] = df[score_col] < low_threshold
    df[f'is_high_{suffix}'] = df[score_col] > high_threshold
    return df

# Apply to each score type
for score_type in ['profitability', 'liquidity', 'solvency', 'leverage_adjusted']:
    dataset1 = flag_anomalies(dataset1, f'score_{score_type}_local', f'{score_type}_local')
    dataset1 = flag_anomalies(dataset1, f'score_{score_type}_global', f'{score_type}_global')

# Combined anomaly detection (low local + low global)
for score_type in ['profitability', 'liquidity', 'solvency', 'leverage_adjusted']:
    dataset1[f'anomaly_low_{score_type}'] = (
        dataset1[f'is_low_{score_type}_local'] & dataset1[f'is_low_{score_type}_global']
    )

# Optional: trend over time (score delta vs previous quarter)
dataset1 = dataset1.sort_values(by=['company', 'quarter'])
for score_type in ['profitability', 'liquidity', 'solvency', 'leverage_adjusted']:
    dataset1[f'trend_{score_type}_local'] = dataset1.groupby('company')[f'score_{score_type}_local'].diff()
    dataset1[f'trend_{score_type}_global'] = dataset1.groupby('company')[f'score_{score_type}_global'].diff()


In [111]:
#bon

In [114]:
import pandas as pd
import streamlit as st

# Charger le fichier
df = pd.read_csv("dataset1_complet.csv")
df = df.sort_values(["company", "quarter"])

# Fonctions
def get_alerts(row):
    alerts = []
    for score in ["profitability", "liquidity", "solvency", "leverage_adjusted"]:
        local = row.get(f"score_{score}_local")
        global_ = row.get(f"score_{score}_global") if score != "profitability" else row.get("score_profitabilty_global")
        if pd.notna(global_) and global_ < 0.2:
            alerts.append(f"Global Low ({score.title()})")
        elif pd.notna(global_) and global_ > 0.8:
            alerts.append(f"Global High ({score.title()})")
        if pd.notna(local) and local > 0.8:
            alerts.append(f"Local Trend ↑ ({score.title()})")
        elif pd.notna(local) and local < 0.2:
            alerts.append(f"Local Drop ↓ ({score.title()})")
    if pd.notna(row.get("revenue_growth")):
        if row["revenue_growth"] > 0.01:
            alerts.append("Rev ↑")
        elif row["revenue_growth"] < -0.01:
            alerts.append("Rev ↓")
    return ", ".join(alerts)

def get_status(row):
    problems = 0
    positives = 0
    for score in ["profitability", "liquidity", "solvency", "leverage_adjusted"]:
        local = row.get(f"score_{score}_local")
        global_ = row.get(f"score_{score}_global") if score != "profitability" else row.get("score_profitabilty_global")
        if pd.notna(local) and pd.notna(global_):
            if local < 0.2 and global_ < 0.2:
                problems += 1
            elif local > 0.8 and global_ > 0.8:
                positives += 1
    if problems >= 2:
        return "Critical"
    elif problems == 1:
        return "Weak"
    elif positives >= 2:
        return "Strong"
    elif positives == 1:
        return "Improving"
    return "Neutral"

# Ajouter colonnes Alert Summary et Status
df["Alert Summary"] = df.apply(get_alerts, axis=1)
df["Overall Status"] = df.apply(get_status, axis=1)
df["Rev Growth"] = df["revenue_growth"].apply(lambda x: f"{x*100:.1f}%" if pd.notna(x) else "")

# Interface Streamlit
st.title("📊 Company Financial Score Dashboard")

# Menu déroulant
company = st.selectbox("Select a company:", sorted(df["company"].unique()))

# Filtrer
df_company = df[df["company"] == company]

# Colonnes à afficher
cols = {
    "score_profitability_local": "Profitability (Local)",
    "score_profitabilty_global": "Profitability (Global)",
    "score_liquidity_local": "Liquidity (Local)",
    "score_liquidity_global": "Liquidity (Global)",
    "score_solvency_local": "Solvency (Local)",
    "score_solvency_global": "Solvency (Global)",
    "score_leverage_adjusted_local": "Adj. Leverage (Local)",
    "score_leverage_adjusted_global": "Adj. Leverage (Global)",
    "Rev Growth": "Rev Growth",
    "Alert Summary": "Alert Summary",
    "Overall Status": "Overall Status"
}

# Affichage
st.subheader(f"📈 Results for {company}")
st.dataframe(df_company[["quarter"] + list(cols.keys())].rename(columns=cols), use_container_width=True)


2025-06-03 09:56:30.970 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 09:56:30.970 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 09:56:30.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 09:56:30.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 09:56:30.977 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 09:56:30.977 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 09:56:30.977 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 09:56:30.977 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [115]:
streamlit run app.py


SyntaxError: invalid syntax (507122745.py, line 1)